# State Vector Prediction in a GlueX Tracking Detector

Useful links
 - https://www.jlab.org/AI/quarterly_problem
 - https://halldweb.jlab.org/talks/ML_lunch/Sep2019/
 - http://www.dannowitz.co/ds-blog/2019/12/02/deep-recurrent-particle-tracking#commenting=

In [1]:
# Importing required libraries

import pandas as pd
import numpy as np

In [20]:
# Imaporting data

MLC_traning_inputs_df = pd.read_csv('MLchallenge2_training.csv')
MLC_traning_df = pd.read_csv('MLchallenge2_training.csv')
test_in_df = pd.read_csv('test_in.csv',header=None, on_bad_lines='skip')
test_prediction_df = pd.read_csv('test_prediction.csv',header=None)

### MLC Traning Inputs Data Set

In [3]:
MLC_traning_inputs_df.head()

,x,y,z,px,py,pz,x1,y1,z1,px1,...,z23,px23,py23,pz23,x24,y24,z24,px24,py24,pz24
0,0.877079,1.32218,65,-0.243980,-0.053204,2.414260,-10.66900,0.330138,176.944,-0.253523,...,341.28,-0.226149,0.106356,2.407600,-27.43270,5.06932,343.405,-0.226602,0.106052,2.407550
1,0.786361,-2.48294,65,0.103229,0.432216,2.592910,7.36649,15.502000,176.944,0.205638,...,341.28,0.326228,0.295249,2.581860,25.26860,37.60720,343.405,0.325574,0.295009,2.581950
2,-13.133900,-26.53090,65,0.064432,-0.020771,0.952952,-7.58617,-30.686700,176.944,0.026643,...,341.28,-0.015729,-0.047137,0.948236,-8.45915,-39.71580,343.405,-0.013650,-0.047591,0.948226
3,18.454200,2.80469,65,-0.019384,0.069384,1.832590,18.04330,6.797470,176.944,0.013039,...,341.28,0.039513,0.051584,1.824120,20.76920,11.70620,343.405,0.039183,0.052164,1.824100
4,15.552100,-19.19600,65,-0.009768,-0.010642,2.366080,15.06810,-19.750200,176.944,-0.014308,...,341.28,-0.013718,-0.002290,2.350870,13.65750,-20.62340,343.405,-0.014211,-0.002519,2.350850


In [4]:
MLC_traning_inputs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194601 entries, 0 to 194600
Columns: 150 entries, x to pz24
dtypes: float64(149), int64(1)
memory usage: 222.7 MB


In [7]:
MLC_traning_inputs_df.equals(MLC_traning_df)
# both data files are identical

True

### Test In Data Set

In [15]:
test_in_df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,0.877,1.322,65.0,-0.244,-0.053,2.414,-10.669,0.330,176.944,-0.254,...,-0.237,0.092,2.407,-22.920,3.053,296.228,-0.236,0.093,2.407,298.354
1,0.786,-2.483,65.0,0.103,0.432,2.593,7.366,15.502,176.944,0.206,...,0.310,0.320,2.581,19.381,32.055,296.228,0.312,0.318,2.581,298.354
2,1.962,2.314,65.0,-0.194,0.088,3.085,-4.705,6.134,176.944,-0.172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-5.597,-21.536,65.0,0.241,0.371,3.249,3.536,-9.556,176.944,0.296,...,0.358,0.245,3.240,296.228,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.030,-0.421,65.0,-0.079,0.042,0.836,-7.952,8.029,176.944,-0.033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
test_in_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4749 entries, 0 to 4748
Data columns (total 91 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4749 non-null   float64
 1   1       4749 non-null   float64
 2   2       4749 non-null   float64
 3   3       4749 non-null   float64
 4   4       4749 non-null   float64
 5   5       4749 non-null   float64
 6   6       4749 non-null   float64
 7   7       4749 non-null   float64
 8   8       4749 non-null   float64
 9   9       4749 non-null   float64
 10  10      4749 non-null   float64
 11  11      4749 non-null   float64
 12  12      4749 non-null   float64
 13  13      4749 non-null   float64
 14  14      4749 non-null   float64
 15  15      4749 non-null   float64
 16  16      4749 non-null   float64
 17  17      4749 non-null   float64
 18  18      4749 non-null   float64
 19  19      4749 non-null   float64
 20  20      4749 non-null   float64
 21  21      4749 non-null   float64
 22  

### Test Prediction Data Set

In [22]:
test_prediction_df.head()

,0,1,2,3,4
0,-23.123945,3.142886,-0.235592,0.091612,2.413377
1,19.633486,32.319292,0.314376,0.316425,2.592952
2,-8.308506,-39.299613,-0.020097,-0.051232,0.948906
3,19.918838,10.664617,0.038102,0.047740,1.864014
4,13.649239,-20.616935,-0.015548,0.001471,2.323953


In [23]:
test_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       10000 non-null  float64
 1   1       10000 non-null  float64
 2   2       10000 non-null  float64
 3   3       10000 non-null  float64
 4   4       10000 non-null  float64
dtypes: float64(5)
memory usage: 390.8 KB
